In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

sales = pd.read_csv("/content/drive/MyDrive/ML project/SalesFINAL12312016.csv", parse_dates=["SalesDate"])
purchase = pd.read_csv("/content/drive/MyDrive/ML project/PurchasesFINAL12312016.csv")
invoice = pd.read_csv("/content/drive/MyDrive/ML project/InvoicePurchases12312016.csv")
end_inv = pd.read_csv("/content/drive/MyDrive/ML project/EndInvFINAL12312016.csv")
beg_inv = pd.read_csv("/content/drive/MyDrive/ML project/BegInvFINAL12312016.csv")
purchase_price = pd.read_csv("/content/drive/MyDrive/ML project/2017PurchasePricesDec.csv")


In [ ]:
display(beg_inv.head())
display(end_inv.head())
nuniques = {"beg_inv": beg_inv.nunique(), "end_inv": end_inv.nunique()}
display(
    pd.DataFrame(nuniques).T[
        [
            "InventoryId",
            "Store",
            "City",
            "Brand",
            "Description",
            "Size",
            "onHand",
            "startDate",
            "endDate",
        ]
    ]
)
print(
    f"beg_inv Brand nunique: {beg_inv.Brand.nunique()}, desc + size nunique: {(beg_inv['Description'] + ' ' + beg_inv['Size']).nunique()}??? Might need cleaning"
)
print(
    f"end_inv Brand nunique: {end_inv.Brand.nunique()}, desc + size nunique: {(end_inv['Description'] + ' ' + end_inv['Size']).nunique()}"
)


In [ ]:
beg_inv_brand = beg_inv.loc[:]
beg_inv_brand["Desc_Size"] = beg_inv_brand["Description"] + " " + beg_inv_brand["Size"]
group_desc = (
    beg_inv_brand[["Brand", "Desc_Size"]].groupby("Desc_Size")["Brand"].unique()
)
group_desc.loc[group_desc.apply(len) > 1]

# ? will assume brand is id for Description + Size


In [ ]:
display(sales.head())
display(
    beg_inv.loc[
        (beg_inv["Brand"] == 1004) & (beg_inv["InventoryId"] == "1_HARDERSFIELD_1004")
    ]
)
# ? Inventory ID = store_city_brand, Brand = description + Size, With Inventory ID we can find how many onhand the inventory have at the beginning and end.
print(sales.Classification.unique())

In [ ]:
display(purchase.head())
display(invoice.head())
# * assume, lead time = Receiving Date - Purchase Order Date. They did not produce their own goods but buy from vendors.
#  * freight is cargo or shipping cost.


In [ ]:
display(purchase_price.head())
purchase_price_vendor = purchase_price["VendorName"].unique()

In [ ]:
from statsmodels.tsa.api import SARIMAX
from IPython.display import clear_output
from math import pow
import plotly.express as px

# * Group by date, sum sales quantity to get total sales quantity per day
sales_quantity_price = sales.groupby("SalesDate").agg({"SalesQuantity": "sum"})

# * Load ARIMA, Autoregressive Moving Average
mod = SARIMAX(sales_quantity_price, order=(5, 1, 1), trend="c")
res = mod.fit()

# * Add more dates to sales_quantity_price, predict until 2016-03-31
for i in pd.date_range("2016-03-01", "2016-03-31", freq="D"):
    sales_quantity_price.loc[i, "SalesQuantity"] = None
sales_quantity_price["forecast"] = res.predict(1, 60)

# * Plot with x-axis limited to January and February
clear_output()
fig = px.line(sales_quantity_price, x=sales_quantity_price.index, y=["SalesQuantity", "forecast"])
fig.update_xaxes(range=["2016-01-01", "2016-02-29"])  # Limiting the x-axis to January and February
fig.show()

# * Calculate MAPE error
absolute_diff = abs(sales_quantity_price["SalesQuantity"] - sales_quantity_price["forecast"])
relative_diff = absolute_diff / sales_quantity_price["SalesQuantity"] * 100
print("\nMAPE:", relative_diff.dropna().mean())

# * Calculate R2
# Calculate the mean of actual values
mean_actual = sales_quantity_price["SalesQuantity"].mean()

# Calculate the total sum of squares (SST)
sst = ((sales_quantity_price["SalesQuantity"] - mean_actual) ** 2).sum()

# Calculate the residual sum of squares (SSR)
ssr = ((sales_quantity_price["SalesQuantity"] - sales_quantity_price["forecast"]) ** 2).sum()

# Calculate R^2
r2 = 1 - (ssr / sst)
print(f'R^2: {r2}')

# Print top 3 highest relative differences
print(relative_diff.sort_values(ascending=False).head(3))

# Calculate and print forecast total demand for March
print("March Forecast Total Demand: ", sales_quantity_price.loc["2016-03-01":"2016-03-31", "forecast"].sum())


In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from sklearn.linear_model import LinearRegression
import pandas as pd

# Assuming 'sales' is your DataFrame
sales_quantity_price = sales.groupby("SalesDate").agg({"SalesQuantity": "sum"})
sales_quantity_price.index = pd.to_datetime(sales_quantity_price.index)
sales_quantity_price['week'] = sales_quantity_price.index.month
sales_quantity_price['day'] = sales_quantity_price.index.dayofweek

# Correcting the frequency to 'M' for month end
fourierM = CalendarFourier(freq="M", order=6)

dp = DeterministicProcess(
    index=sales_quantity_price.index,
    constant=True,
    order=4,
    seasonal=True,
    additional_terms=[fourierM],
)

X = dp.in_sample()
model = LinearRegression().fit(X, sales_quantity_price['SalesQuantity'])
y_pred = pd.Series(
    model.predict(X),
    index=X.index,
    name="fitted"
)

# Filter for positive forecast values
sales_quantity_price["forecast"] = y_pred
filtered_sales_quantity_price = sales_quantity_price[sales_quantity_price["forecast"] > 0]

# Plotting only positive forecasts
filtered_sales_quantity_price[["SalesQuantity", "forecast"]].plot()

# * calculate MAPE error for positive forecasts only
absolute_diff = abs(filtered_sales_quantity_price["SalesQuantity"] - filtered_sales_quantity_price["forecast"])
relative_diff = absolute_diff / filtered_sales_quantity_price["SalesQuantity"] * 100
print("\nMAPE:", relative_diff.dropna().mean())

# * Calculate R2 for positive forecasts only
# Calculate the mean of actual values
mean_actual = filtered_sales_quantity_price["SalesQuantity"].mean()

# Calculate the total sum of squares (SST)
sst = ((filtered_sales_quantity_price["SalesQuantity"] - mean_actual) ** 2).sum()

# Calculate the residual sum of squares (SSR)
ssr = ((filtered_sales_quantity_price["SalesQuantity"] - filtered_sales_quantity_price["forecast"]) ** 2).sum()

# Calculate R^2
r2 = 1 - (ssr / sst)
print(f'R^2: {r2}')

# Print top 3 highest relative differences
print(relative_diff.sort_values(ascending=False).head(3))

# Calculate and print forecast total demand for March
march_forecast_total_demand = filtered_sales_quantity_price.loc["2016-03-01":"2016-03-31", "forecast"].sum()
print("March Forecast Total Demand: ", march_forecast_total_demand)


In [ ]:
  ABC = (
    sales.groupby("Brand")
    .agg({"SalesQuantity": "sum"})
    .sort_values("SalesQuantity", ascending=False)
)
display(
    "Top 5 Demand",
    ABC.head(),
    "Bottom 5 Demand",
    ABC.tail(),
    "describe data",
    ABC.describe().T,
)
bins = [0, 100, 1000, 30000]
labels = ["C: <100", "B: 100-1k", "A: 1k-30k"]
ABC['bin'] = pd.cut(ABC["SalesQuantity"], bins, labels=labels)

In [ ]:
plt.bar(ABC['bin'].value_counts().index, ABC['bin'].value_counts().values, color=['blue','green','red'])
for i, value in enumerate(ABC['bin'].value_counts().values):
    plt.text(i, value, str(value), ha='center', va='bottom')
plt.title('Histogram of ABC analysis, total unique brand from group A to C')
plt.xlabel('Group')
plt.ylabel('Frequency')

In [ ]:
total_group = ABC.groupby('bin').agg({"SalesQuantity":'sum'})['SalesQuantity']
total = px.histogram(total_group.value_counts(), x=total_group.index, y=total_group.values, color=total_group.index, title="ABC analysis, total sales of brand from group A to C",text_auto=True)
total.update_layout(
  width=800,
  height=600,
    xaxis_title='Group',
    yaxis_title='Total_sales',
    bargap=0.1,  # Gap between bars
)
total.show()

In [ ]:
# * Demand 2 month
from math import pow, ceil


demand_jan_feb = (

    sales.groupby("Brand")

    .agg({"SalesQuantity": "sum"})

    .sort_values("SalesQuantity", ascending=False)
)


# * Cost Per Order (CPO), Freight Cost

purchase_invoice = pd.merge(
    left=purchase[["PONumber", "Brand", "Description", "Size", "Quantity"]],
    right=invoice,
    on="PONumber",
    how="right",
)

purchase_invoice_vol = pd.merge(
    left=purchase_invoice,
    right=purchase_price[["Brand", "Volume"]],
    on="Brand",
    how="left",
)
purchase_invoice_vol["vol_quantity"] = purchase_invoice_vol["Quantity_x"].astype(
    "float"
) * purchase_invoice_vol["Volume"].astype("float")
purchase_invoice_vol.set_index("PONumber", inplace=True)
purchase_invoice_vol["vol_quantity_total"] = purchase_invoice_vol.groupby(
    "PONumber"
).agg({"vol_quantity": "sum"})
purchase_invoice_vol["CPO"] = (
    purchase_invoice_vol["vol_quantity"]
    / purchase_invoice_vol["vol_quantity_total"]
    * purchase_invoice_vol["Freight"]
)
# **************************************
CPO = purchase_invoice_vol.groupby("Brand").agg({"CPO": "mean"})
#****************************************
# * Cost Per Unit (C), Holding Cost (%) assume holding cost = 30%, Holding Cost ($) = I*C = I*0.3
C = purchase_price[["Brand", "PurchasePrice"]].set_index("Brand")
H = C * 0.3

# * Combine to Brand
EOQ = demand_jan_feb.copy()
EOQ["CPO"] = CPO
EOQ["H"] = H

# * Calculate and dropna
EOQ["EOQ"] = 2 * EOQ["SalesQuantity"] * EOQ["CPO"] / EOQ["H"]
EOQ["EOQ"] = EOQ["EOQ"].apply(lambda x: ceil(pow(x, 0.5)) if pd.notna(x) else 0)
EOQ.dropna(inplace=True)
EOQ["Volume"] = purchase_price[["Brand", "Volume"]].set_index("Brand")
EOQ['ABC']=ABC['bin']
display(EOQ.head(15), EOQ.describe(),EOQ.groupby("ABC").agg({"EOQ":"mean"}).T)

In [ ]:
# * Average Daily Unit Sales
sales_velocity = (
    sales.groupby(["Brand", "Description"]).agg({"SalesQuantity": "sum"}).reset_index()
)
days = (
    pd.to_datetime(sales["SalesDate"]).max() - pd.to_datetime(sales["SalesDate"]).min()
).days
sales_velocity["mean_daily_sales"] = sales_velocity["SalesQuantity"] / days
mean_daily_sales = sales_velocity.set_index("Brand")["mean_daily_sales"]
display(mean_daily_sales.to_frame().T, mean_daily_sales.describe().to_frame().T)

In [ ]:
# * Average Lead Time
purchase["lead_time"] = (
    pd.to_datetime(purchase["ReceivingDate"]) - pd.to_datetime(purchase["PODate"])
).dt.days
lead_time_brand = purchase.groupby("Brand").agg({"lead_time": "mean"})
display(lead_time_brand.T, lead_time_brand.describe().T)

In [ ]:
# * Safety Stock
max_sales_brand = sales.groupby("Brand").agg({"SalesQuantity": "max"})
safety_stock = (
    max_sales_brand["SalesQuantity"]
    - sales_velocity.set_index("Brand")["mean_daily_sales"]
)
display(safety_stock.to_frame().T, safety_stock.describe().to_frame().T)
# * notice there are negatives to fix this problem we will just set them to 0
safety_stock[safety_stock < 0] = 0

In [ ]:
# * Reorder Point Analysis
RPA = (lead_time_brand['lead_time'] * mean_daily_sales) + safety_stock
RPA.name = "ReorderPoint"
RPA.dropna(inplace=True)
RPA = RPA.apply(lambda x: ceil(x)).to_frame()
RPA['ABC'] = ABC['bin']
display(RPA.T,RPA.describe().T)

In [ ]:
fig = px.histogram(
    RPA, x="ReorderPoint", color="ABC", nbins=50, title="Reorder Point Analysis distribution", log_y=True
)
fig.update_layout(
    bargap=0.2,
    xaxis_title_text="ReorderPointBin",  # xaxis label
    yaxis_title_text="total_count_brand",  # yaxis label

)
fig.show()

In [ ]:
fig = px.histogram(lead_time_brand, x='lead_time', nbins=50, title="Lead Time Analysis distribution",text_auto=True)
fig.update_layout(
    bargap=0.1,
    xaxis_title_text="average_lead_time",  # xaxis label
    yaxis_title_text="total_count_brand",  # yaxis label
    height=600
)
fig.show()

In [ ]:
import plotly.graph_objects as go

annual_inventory_value = end_inv.copy()
annual_inventory_value["total_value"] = (
    annual_inventory_value["onHand"] * annual_inventory_value["Price"]
)
brand_inventory_value = annual_inventory_value.groupby("Brand").agg(
    {"Price": "first", "Description": "first", "total_value": "sum", "onHand": "sum"}
)

brand_inventory_value.sort_values("total_value", ascending=False, inplace=True)
brand_inventory_value["RPA"] = RPA["ReorderPoint"]
brand_inventory_value["total_value_if_RPA"] = (
    brand_inventory_value["RPA"] * brand_inventory_value["Price"]
)
brand_inventory_value["total_value_saved_if_RPA"] = (
    brand_inventory_value["total_value"] - brand_inventory_value["total_value_if_RPA"]
)
brand_inventory_value.dropna(inplace=True)

fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=brand_inventory_value["total_value"],
        name="end of year inventory",
        nbinsx=50,
    )
)
fig.add_trace(
    go.Histogram(
        x=brand_inventory_value["total_value_if_RPA"],
        name="if follow reorder point/optimal level",
        nbinsx=50,
    )
)
fig.update_yaxes(type="log")
fig.update_layout(
    title_text="Total Inventory Value Distribution",
    xaxis_title_text="total inventory value",  # xaxis label
    yaxis_title_text="total number of brands",  # yaxis label
    bargap=0.2,  # gap between bars of adjacent location coordinates
    bargroupgap=0.1,  # gap between bars of the same location coordinates
)

fig.show()

display(brand_inventory_value[['total_value','total_value_if_RPA','total_value_saved_if_RPA']].sum().apply(lambda x: f"${x:,.0f}").to_frame().T)
brand_inventory_value["total_value"] = brand_inventory_value["total_value"].apply(
    lambda x: f"${x:,.0f}"
)
brand_inventory_value["total_value_if_RPA"] = brand_inventory_value[
    "total_value_if_RPA"
].apply(lambda x: f"${x:,.0f}")
brand_inventory_value["total_value_saved_if_RPA"] = brand_inventory_value[
    "total_value_saved_if_RPA"
].apply(lambda x: f"${x:,.0f}")
display(brand_inventory_value.head(10), brand_inventory_value.tail(10))
